# 158.755 Data Science - Making Sense of Data Project 4
# Study In Recommender Engine

## 1.0 Abstract

A recommender system, or a recommender engine intends to seek prediction of 'rating' or 'preference' for a user that would rank an item or event, mainly used in commercial web applications. 

They have been utilized in various areas over the last ten years, commonly used as playlist generators for video and music services such as Youtube, Netflix, and BiliBili, product recommenders like Amazon, Trademe, and Ebay, and content or news based recommenders for social media platforms as Facebook, Twitter and Instergram. Also, there are other popular recommender systems being utilized for specific topics as hotels booking, dating matching, and online competition game team up.

Flow and monetization are the core concept associated with commercial web applications for internet. Simply speaking, flow is the measurement that the number of visitings for a web application, while monetization evaluates its overall income for keeping the business up and running continuously and healthily. (i.e. advertisement income of Youtube, membership subscriptions for Netflix, and fees/charges for each successfully sale for Amazon ). 

Therefore, recommendation engine is one of the key part for monetization that allows finding consumers' real demand through directing them to their most interest items or services. In addition to this , owners of commercial web applications, would be able accurately deploy advertisements and services to their customers based on a successful recommendation engine set up, for example ,playlist generators (Youtube) could generate just right advertisements to their viewers , and product recommenders like Amazon could pushed customers related products when they are viewing a certain item. Thus, it would optimise the resource usage and amplify income for a commercial organisation who has developed such kind of commercial web applications. 

In this project , we would implement a ***'breath first'*** strategy so as to explore as much lifecycle and implementations for a recommendation engine , such as the input and output data structure, machine learning algorithms, evaluation standard for comparison of different algorithms, and its position and role in architecture of a commercial web applications , instead of undertaking ***in-depth*** research on specific algorithms as per their theory and implementations. Thus, we will simply use an existing library to compute our findings and recommendations for this project.



## 2.0 Indroduction


## 3.0 Data Collection

The datasets for a commercial web application have almost never been exposed for public usage or research, as they are private and strategy assets for business. Normally they are generated through 

Fortunately, there are existing web source regarding recommendation engine study for us to play with , which is Amazon product data prepared by Associate Professor Julian McAuley, UCSD at http://jmcauley.ucsd.edu/data/amazon/.

Normally, user activates for a commercial web app could be concluded as viewing a certain web page , purchasing products, comments and rank items , content or news etc. It is quite difficult to summarise a standard data structure for a recommendation engine due to the variety demands and requirements for various business activities.

However, a classic data structure format has been widely used after long term experimentation and operation with major internet giant companies such as Amazon and Facebook:

- ***user id*** : unique user id
- ***item id*** : unique item id
- ***behaviour type*** : type of behaviour , i.e. purchase or view an item
- ***context*** : behaviour context, including location and time etc.
- ***behaviour weight*** : weight could be the viewing length for a video or rank for an item 
- ***behaviour content*** : if a user comments something, the content could be saved as a text file. If user click an item, the content could be a binary input.


### 3.1 Web API

Amazon Instant Video review data , http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Amazon_Instant_Video_5.json.gz , has been selected as the dataset for this project. 




### 3.2 Web Crawling

## 4.0 EDA

## 5.0 Algorithm Research

As there are many approaches to implement recommender engine, such as Collaborative filtering, Content-based filtering, Multi-criteria recommender systems etc. Their brief concept and implementations could be refer to 
https://en.wikipedia.org/wiki/Recommender_system.

In this project , we would use an existing library named as ***SurPRISE***, stands for ***Simple Python RecommendatIon System Engine.***,http://surpriselib.com/.

As it has provided various ready-to-use prediction algorithms with good documentation and use cases, such as baseline algorithms, neighborhood methods, matrix factorization-based ( SVD, PMF, SVD++, NMF), and many others. Also, various similarity measures (cosine, MSD, pearson…) are built-in.

In this section ,we will mainly focus on ***matrix factorization-based modules and similarity modules***.


### 5.1 Input Data Structure

The input data structure is very simple , just a big user - item with rating matrix , that is represented as a specific user ranks an item with certain rating as per the code below:

In [3]:
#full dataset for research
import numpy as np
import pandas as pd
df = pd.read_csv('full_dataset.csv')
# df = pd.read_csv('testForInput.csv')
# df = pd.read_csv('test.csv')
print(df.shape)
# df = df[:1000]
df.shape

(497577, 3)


(497577, 3)

In [4]:
df.head()

,userID,itemID,rating
0,A1HP7NVNPFMA4N,0700026657,5
1,A1JGAP0185YJI6,0700026657,4
2,A1YJWEXHQBWK2B,0700026657,3
3,A2204E1TH211HT,0700026657,2
4,A2RF5B5H74JLPE,0700026657,5


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497577 entries, 0 to 497576
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   userID  497577 non-null  object
 1   itemID  497577 non-null  object
 2   rating  497577 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 11.4+ MB


### 5.2 Performance measures

The commonly used metrics are the ***Mean Absolute Error(MAE)*** and ***Root Mean Squared Error(RMSE)***,  ***precision*** and ***recall*** will also be used to evaluate the quality of a model for comparison. 


##  6.0 Matrix Factorization-Based Modules

In [6]:
from collections import defaultdict
from surprise import Dataset
import pandas as pd
from surprise import SVD
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
import numpy as np
from surprise import dump
import os
from surprise.model_selection import KFold
import io  # needed because of weird encoding of u.item file

from surprise import KNNBaseline
from surprise import get_dataset_dir
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import GridSearchCV

def precision_recall_at_k(predictions, k=10, threshold=3.5):
    '''Return precision and recall at k metrics for each user.'''

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    return precisions, recalls




In [7]:
# df = pd.read_csv('full_dataset.csv')
# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)

### 6.1 SVD

The famous SVD algorithm, which was popularized by Simon Funk during the Netflix Prize in 2006. It's documentation and reference could be referred to links as below:

https://sifter.org/~simon/journal/20061211.html

https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD

SVD is stand for singular value decomposition mathmatically, is a factorization of a real or complex matrix that generalizes the eigendecomposition of a square normal matrix to any m × n matrix via an extension of the polar decomposition. 

Actually , it is dry and headache to focus on the mathmatical details for this algorithm, ***SurPRISE*** lib has encapsulated ready to use functions to implement this approach.

The code below is a standard machine learning process , which has iterated through all combinations of parameters in ***param_grid*** variable with K folds method (3 folds), so as to find the best prediction model based on RMSE and MAE score.

***{'n_epochs': 15, 'lr_all': 0.01, 'reg_all': 0.2}*** parameters have been found as the best score model for this approach


In [12]:
param_grid = {'n_epochs': [5, 10,15], 'lr_all': [0.002, 0.005,0.01],
              'reg_all': [0.2, 0.4, 0.6]}

In [112]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import GridSearchCV


#choose various parameters for model
param_grid = {'n_epochs': [5, 10,15], 'lr_all': [0.002, 0.005,0.01],
              'reg_all': [0.2, 0.4, 0.6]}

#iterate thorugh parameter grid to find best model
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(data)

# best RMSE score
print('best RMSE score')
print(gs.best_score['rmse'])
print(gs.best_score['mae'])
# combination of parameters that gave the best RMSE score
print('combination of parameters that gave the best RMSE score')
print(gs.best_params['rmse'])
print(gs.best_params['mae'])

best RMSE score
1.0272657422404547
0.7620971595317978
combination of parameters that gave the best RMSE score
{'n_epochs': 15, 'lr_all': 0.01, 'reg_all': 0.2}
{'n_epochs': 15, 'lr_all': 0.01, 'reg_all': 0.2}


Once the best SVD model has been selected , the ***precision@k and recall@k*** could be computed with 5 folds method. As their results are very close, we could justify that data are distributed with not much outliers.

In [113]:
kf = KFold(n_splits=5)
#best algo based on above
algo = gs.best_estimator['rmse']
#print precision@k and recall@k
for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4)

    # Precision and recall can then be averaged over all users
    print('precision@k')
    print(sum(prec for prec in precisions.values()) / len(precisions))
    print('recall@k')
    print(sum(rec for rec in recalls.values()) / len(recalls))

precision@k
0.9042092753091527
recall@k
0.8118487226103486
precision@k
0.9047240901823097
recall@k
0.8099018479126098
precision@k
0.9045551391411665
recall@k
0.8109153981821217
precision@k
0.9043249646528346
recall@k
0.8139062717987631
precision@k
0.9055428117019171
recall@k
0.8126901223660934


We can use the code below to save this model for future reuse.

In [114]:
#save the model for later use
dump.dump(os.path.expanduser("SVD_GS_BEST_RMSE"), algo=gs.best_estimator['rmse'])

# matrix_factorization.SVDpp

In [10]:
from surprise import SVDpp

In [13]:
gs = GridSearchCV(SVDpp, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(data)

# best RMSE score
print('best RMSE score')
print(gs.best_score['rmse'])
print(gs.best_score['mae'])
# combination of parameters that gave the best RMSE score
print('combination of parameters that gave the best RMSE score')
print(gs.best_params['rmse'])
print(gs.best_params['mae'])

# algo = NMF()
# cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
kf = KFold(n_splits=5)
#best algo based on above
algo_SVDpp = gs.best_estimator['rmse']
#print precision@k and recall@k
for trainset, testset in kf.split(data):
    algo_SVDpp.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4)

    # Precision and recall can then be averaged over all users
    print('precision@k')
    print(sum(prec for prec in precisions.values()) / len(precisions))
    print('recall@k')
    print(sum(rec for rec in recalls.values()) / len(recalls))

best RMSE score
1.0289773246775415
0.7621566029433042
combination of parameters that gave the best RMSE score
{'n_epochs': 15, 'lr_all': 0.01, 'reg_all': 0.2}
{'n_epochs': 15, 'lr_all': 0.01, 'reg_all': 0.2}
precision@k
0.9430892627321197
recall@k
0.7906440639449057
precision@k
0.9422059919053221
recall@k
0.7907455676858666
precision@k
0.944763022451312
recall@k
0.7885706254019044
precision@k
0.9418632628431306
recall@k
0.7898403311048827
precision@k
0.9423102507053235
recall@k
0.7892250744462574


In [14]:
dump.dump(os.path.expanduser("SVDpp_GS_BEST_RMSE"), algo=gs.best_estimator['rmse'])

## 7.0 Similarity Modules

### 7.1 Cosine 

Since there are not many parameters to be tested for a similarity module, we can simple set up model and cross validate them with K folds, the code below computes the RMSE and MAE result plus their fir and test running time.
As RMSE and MAE results are similar for 5 folds, we can specify that there are not much skewness for the dataset.

The documentation for Cosine Similarity could be refered to https://surprise.readthedocs.io/en/stable/similarities.html.

In [8]:
sim_options = {'name': 'cosine', 'user_based': False} # or item based
algo = KNNBaseline(sim_options=sim_options)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0862  1.0859  1.0781  1.0855  1.0827  1.0837  0.0030  
MAE (testset)     0.7404  0.7391  0.7351  0.7391  0.7369  0.7381  0.0019  
Fit time          39.62   41.71   49.48   38.45   40.03   41.86   3.95    
Test time         7.05    6.45    5.80    5.75    6.06    6.22    0.48    


{'test_rmse': array([1.08620342, 1.08590904, 1.07812575, 1.08548604, 1.08273486]),
 'test_mae': array([0.74041087, 0.73912286, 0.73505521, 0.73914556, 0.73690182]),
 'fit_time': (39.62233304977417,
  41.70804286003113,
  49.47762894630432,
  38.45476675033569,
  40.02991199493408),
 'test_time': (7.045154809951782,
  6.44753098487854,
  5.803295135498047,
  5.749550104141235,
  6.057494163513184)}

We can also view their ***precision@k and recall@k*** could be computed with 5 folds. Also their results are very close.

In [119]:
kf = KFold(n_splits=5)
#best algo based on above
sim_options = {'name': 'cosine', 'user_based': False} # or item based
algo = KNNBaseline(sim_options=sim_options)
#print precision@k and recall@k
for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4)

    # Precision and recall can then be averaged over all users
    print('precision@k')
    print(sum(prec for prec in precisions.values()) / len(precisions))
    print('recall@k')
    print(sum(rec for rec in recalls.values()) / len(recalls))

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
precision@k
0.8961830077348748
recall@k
0.792255998065553
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
precision@k
0.8966896498889148
recall@k
0.7901736035234929
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
precision@k
0.8975790343687602
recall@k
0.788469409113767
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
precision@k
0.8959885397779883
recall@k
0.7931027252017093
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
precision@k
0.8982012734753935
recall@k
0.788705855925141


Lastly , save the model on local disk for future reuse.

In [120]:
#save the model for later use
dump.dump(os.path.expanduser("KNN_COSINE"), algo=algo)

# KNNWithMeans

In [15]:
from surprise import KNNWithMeans #

In [16]:
sim_options = {'name': 'cosine', 'user_based': False} # or item based
algo = KNNWithMeans(k=40, min_k=1,sim_options=sim_options) 
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1062  1.1044  1.1056  1.1098  1.1064  1.1065  0.0018  
MAE (testset)     0.7556  0.7540  0.7545  0.7595  0.7584  0.7564  0.0022  
Fit time          49.68   41.42   40.79   53.42   57.10   48.48   6.47    
Test time         5.49    5.71    6.07    6.74    6.10    6.02    0.43    


{'test_rmse': array([1.10618382, 1.10443725, 1.10556729, 1.10976373, 1.10643406]),
 'test_mae': array([0.75555747, 0.75400287, 0.75446297, 0.75954783, 0.75838677]),
 'fit_time': (49.68032908439636,
  41.42016577720642,
  40.78865575790405,
  53.41527223587036,
  57.098381996154785),
 'test_time': (5.485365867614746,
  5.706478118896484,
  6.074939966201782,
  6.73696494102478,
  6.098551988601685)}

In [17]:
kf = KFold(n_splits=5)
#best algo based on above
sim_options = {'name': 'cosine', 'user_based': False} # or item based
algo = KNNWithMeans(k=40, min_k=1,sim_options=sim_options) 
#print precision@k and recall@k
for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4)

    # Precision and recall can then be averaged over all users
    print('precision@k')
    print(sum(prec for prec in precisions.values()) / len(precisions))
    print('recall@k')
    print(sum(rec for rec in recalls.values()) / len(recalls))

Computing the cosine similarity matrix...
Done computing similarity matrix.
precision@k
0.8939064725067764
recall@k
0.7758361322372259
Computing the cosine similarity matrix...
Done computing similarity matrix.
precision@k
0.8932811743829661
recall@k
0.7753662129344113
Computing the cosine similarity matrix...
Done computing similarity matrix.
precision@k
0.8926267351556308
recall@k
0.7781896418478791
Computing the cosine similarity matrix...
Done computing similarity matrix.
precision@k
0.8918243682379483
recall@k
0.7752623755221357
Computing the cosine similarity matrix...
Done computing similarity matrix.
precision@k
0.8920565436524373
recall@k
0.7775257432824312


In [18]:
#save the model for later use
dump.dump(os.path.expanduser("KNNWithMeans_cosine"), algo=algo)

# SlopeOne

In [19]:
from surprise import SlopeOne

In [20]:
algo = SlopeOne()
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SlopeOne on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1355  1.1314  1.1321  1.1331  1.1393  1.1343  0.0029  
MAE (testset)     0.7590  0.7520  0.7553  0.7540  0.7600  0.7561  0.0030  
Fit time          19.79   19.11   17.37   18.91   21.15   19.27   1.23    
Test time         5.53    4.88    5.41    5.08    4.69    5.12    0.32    


{'test_rmse': array([1.13550877, 1.13144892, 1.13210966, 1.13305882, 1.13928786]),
 'test_mae': array([0.75903826, 0.75198709, 0.75527358, 0.75398389, 0.75999962]),
 'fit_time': (19.79092502593994,
  19.11101007461548,
  17.365437030792236,
  18.911175966262817,
  21.147334098815918),
 'test_time': (5.526959180831909,
  4.880213022232056,
  5.414744138717651,
  5.080150127410889,
  4.6894800662994385)}

In [21]:
kf = KFold(n_splits=5)
#best algo based on above
algo = SlopeOne()
#print precision@k and recall@k
for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4)

    # Precision and recall can then be averaged over all users
    print('precision@k')
    print(sum(prec for prec in precisions.values()) / len(precisions))
    print('recall@k')
    print(sum(rec for rec in recalls.values()) / len(recalls))

precision@k
0.907304304534314
recall@k
0.7576945504083302
precision@k
0.9068103112171201
recall@k
0.7554976560210354
precision@k
0.9089315257352952
recall@k
0.7578406741491446
precision@k
0.9094872383576333
recall@k
0.7536212582420128
precision@k
0.9072505249283516
recall@k
0.7548103987324362


In [22]:
dump.dump(os.path.expanduser("SlopeOne_MSD"), algo=algo)

# CoClustering

In [8]:
from surprise import CoClustering

In [9]:
algo = CoClustering()
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm CoClustering on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0791  1.0776  1.0776  1.0808  1.0773  1.0785  0.0013  
MAE (testset)     0.7282  0.7271  0.7274  0.7292  0.7257  0.7275  0.0012  
Fit time          23.31   23.35   22.95   22.92   25.14   23.53   0.82    
Test time         1.15    1.14    1.13    0.99    1.00    1.08    0.07    


{'test_rmse': array([1.07908923, 1.07764887, 1.07762735, 1.08081944, 1.07728414]),
 'test_mae': array([0.72819108, 0.72708234, 0.72736888, 0.72915683, 0.72565143]),
 'fit_time': (23.31475591659546,
  23.348692893981934,
  22.952141046524048,
  22.9163761138916,
  25.140856981277466),
 'test_time': (1.14695405960083,
  1.1443917751312256,
  1.1290650367736816,
  0.9937000274658203,
  1.0017008781433105)}

In [10]:
kf = KFold(n_splits=5)
#best algo based on above
algo = CoClustering()
#print precision@k and recall@k
for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=4)

    # Precision and recall can then be averaged over all users
    print('precision@k')
    print(sum(prec for prec in precisions.values()) / len(precisions))
    print('recall@k')
    print(sum(rec for rec in recalls.values()) / len(recalls))

precision@k
0.9174858179646777
recall@k
0.7462795498704449
precision@k
0.9202601719197723
recall@k
0.7426032055153045
precision@k
0.9196174135409743
recall@k
0.7472589050036618
precision@k
0.9188577445912317
recall@k
0.7494450486868863
precision@k
0.9192650828264296
recall@k
0.7427458714847792


In [11]:
dump.dump(os.path.expanduser("CoClustering_cosine"), algo=algo)

## 8.0 Top N Recommender 

### 8.1 Get Top N Items 

This approach is based on the the top-10 items with highest rating prediction for each user in the dataset. The input could be user rating to different itmes (i.e. 20 items), then , it will return top 10 best prediction items.

Below is a simple example showing the input and output for the top 10 items recommended for a user based on its prediction model

In [ ]:
def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

### 8.2 Get the k nearest neighbors of a user (or item)

We can use the get_neighbors() methods of the algorithm object. This is only relevant for algorithms that use a ***similarity measure***, such as the k-NN algorithms.

Below is an example where we retrieve the 10 nearest neighbors of one of the video games from the video game review dataset. The output is represneted as 10 rows from the dataset.


In [ ]:
algo.get_neighbors(int(10000), k=10)

## 9.0 Presentation of Web App

## 10.0 Recommendation Engine in System Architecture 